# 读取fasta作为输入序列

In [ ]:
# 读取fasta文件作为字符串
def read_fasta(file_path):
    with open(file_path, 'r') as file:
        sequences = {}
        seq_name = None
        seq = ''
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if seq_name:
                    sequences[seq_name] = seq
                seq_name = line[1:]  # 或者使用 line.split()[0][1:] 获取序列名称
                seq = ''
            else:
                seq += line
        if seq_name:
            sequences[seq_name] = seq
    return sequences



In [ ]:
# 使用示例
fileA = 'sequence (13.).fasta'

sequences = read_fasta(fileA)
print(sequences)

In [ ]:
A = sequences['AF485760.1 Aeromonas hydrophila strain AHK1 aerolysin (aerA) gene, partial cds']

In [ ]:
fileA = 'sequence (14.).fasta'
sequences = read_fasta(fileA)
print(sequences)

In [ ]:
B = sequences['AF485759.1 Aeromonas hydrophila strain AHBHP3 aerolysin (aerA) gene, partial cds']

# 草稿部分

In [ ]:
%%time
import numpy as np
import pandas as pd
# 伪代码中的参数
a = -2  # gap 罚分 #这个例子是我手算的例子，用来修正算法
b = -1  # mismatch 罚分 #这个例子是我手算的例子，用来修正算法
c = 2  # match 奖励 #这个例子是我手算的例子，用来修正算法

'''
    a = -2  # gap 罚分
    b = -1  # mismatch 罚分
    c = 3  # match 奖励
'''

# 伪代码中的数据
#A = "ATCGGGCTACATCGGGCTACGGATCGGGCTACGAAAAAAAAAAAAAAAAAAAAAA"
#B = "ATCGGATCGGGCTACGATCGGGCTACGATCGCGTTTTTTTAAAAAGAAAAAAAAGGGGGGGGTGTATTGTA"
#A = "GATTACA" #这个例子是我手算的例子，用来修正算法
#B = "GCATGCG" #这个例子是我手算的例子，用来修正算法
#A = "ATCGGGCTACATCGGGCTACGGATCGGGCTACA"
#B = "ATCGGATCGGGCTACGATCGGGCTACGATCGCG"

len_A = len(A)
len_B = len(B)
# 路径变量
pathWays = []
# 路径变量的初始值，矩阵的右下角是
OrignLocus = (len(A), len(B))
# 存储结果
results = []
# 创建大小为（len(A) + 1） x （len(B) + 1）的矩阵
matrix = np.zeros((len_A + 1, len_B + 1))
#matrix = pd.DataFrame(np.zeros((len_A + 1, len_B + 1)), index=['-']+list(A), columns=['-']+list(B))

# 初始化第一行和第一列
for i in range(len_A + 1):
    matrix[i][0] = i * a

for j in range(len_B + 1):
    matrix[0][j] = j * a

# 计算打分值
for i in range(1, len_A + 1, 1):
    # i的取值范围是从1到len(A)+1，在这个例子里是0-6
    for j in range(1, len_B + 1, 1):
        # 计算矩阵(i,j)处的值
        if A[i-1] != B[j-1]:
            # 如果不相等，就是mismatch
            matrix[i][j] = max(matrix[i-1][j] + a, matrix[i][j-1] + a, matrix[i-1][j-1] + b)
        else:
            # 如果相等，就是match
            matrix[i][j] = max(matrix[i-1][j] + a, matrix[i][j-1] + a, matrix[i-1][j-1] + c)
# matrix
# 打印矩阵
print(matrix)


In [ ]:
def calculate_direction(locus):# 基于当前坐标计算方向
    # 方向，每次使用都更新
    i = locus[0]
    j = locus[1]
    calculateDirections = []
    print(i,j)
    # 原来的写法没考虑match或者mismatch的情况，如果matchl，那么一定会产生一个方向2，即calculateDirections.extend('2')
    print(A[i-1],B[j-1])
    if A[i-1] == B[j-1]:
        # 如果相等，就是match
        print(A[i-1],B[j-1])
        calculateDirections.extend('2') # 相等的时候一定可以走对角线
        print('match')
        pass
    else:
        if matrix[i-1][j-1].size > 0 and np.any(matrix[i-1][j-1] + b == matrix[i][j]):
            calculateDirections.extend('2')
        print('mismatch')
    # 检查对应位置的序列A得值是否等于序列B的值
    # 检查左方  左是0
    if matrix[i-1][j].size > 0 and matrix[i-1][j] + a == matrix[i][j]:
        calculateDirections.extend('0')
        print('左移')
    # 检查上方  上是1
    if matrix[i][j-1].size > 0 and matrix[i][j-1] + a == matrix[i][j]:
        calculateDirections.extend('1')
    print(calculateDirections)
    return calculateDirections

In [ ]:
# 从pathWays计算新坐标，获得新坐标表列表，第二次计算
def calculate_new_locus(pathWays):
    new_locuss=[]
    Directions = len(pathWays)
    print(f"{'从上游有'}{Directions}{'条路径需要计算'}")
    for path in range(0, Directions): # 索引传入的值（‘方向0’或‘方向1’或‘方向2’）在pathWays中的条目，
        lastDirection = pathWays[path].split(',',-1)
        zero = lastDirection.count('0')# 左移
        one = lastDirection.count('1')# 上移
        two = lastDirection.count('2')# 左上移
        locuss = (int(OrignLocus[0]) - zero - two, int(OrignLocus[1]) - one - two)
        new_locuss.append(locuss)
    print(f"{'从pathway新计算的坐标集合'}{new_locuss}")
    return new_locuss

In [ ]:
# 第二次坐标计算
def second_calculate_direction(new_locuss,pathWays):
    pWs = len(new_locuss)#这个时候len(new_locuss)一定等于len(pathWays)
    print(f"{'Total'}{pWs}{'个new locuss'}")#计算结果
    localDirectionss = []
    addPathways= []
    for pW in range(0,pWs):
        # 计算新坐标中的每一个值的方向
        locus_tuple = new_locuss[pW]
        if locus_tuple == (0,0):# locus_tuple坐标为（0，0）的时候，不需要计算方向，直接把这条pathWay加到result中的条目后面。
            results.append(pathWays[pW])
            print(f"{'到达终点后pathway的第'}{pW}{'个'}{'路径写入result'}")#计算结果
            continue #跳出本次循环，进入下一次循环
        else:# 不到终点就继续计算
            localDirections = calculate_direction(locus_tuple)#计算新方向，输出0，1，2，会输出不止一个新方向，
            print(f"{'新方向有'}{localDirections}")#展示计算结果
            for num in range(0,len(localDirections)):
                addPathway = pathWays[pW] + ',' + f"{localDirections[num]}"#形成新的路径
                addPathways.append(addPathway)
            #addPathway = pathWays[pW] + ',' + f"{localDirections[0]}"#形成新的路径
            #print(f"{'new pathway'}{addPathway}")#展示新路径
            #addPathways.append(addPathway)#把路径添加到路径变量
            #print(f"{'addPathways'}{addPathways}")#展示路径变量
        #localDirections = calculate_direction(locus_tuple)
        #localDirectionss.append(localDirections)
        #print(localDirectionss)# localDirectionss 在这里是一个列表，列表中的每一个元素都是一个存储了每个方向单独计算的新方向的列表，所以有可能是套娃列表，也有可能是单层列表。
    pathWays = addPathways
    print(f"{'pathWays的计算结果'}{pathWays}")#计算结果
    return pathWays


In [ ]:
# 第一次计算
localDirections = calculate_direction(OrignLocus)
R = len(localDirections)
for i in range(R):
    pathway = f"{localDirections[i]}"
    pathWays.append(pathway)
#localDirections = []
#pathWays

In [ ]:
while len(pathWays) != 0:
    new_locuss = calculate_new_locus(pathWays)
    pathWays = second_calculate_direction(new_locuss, pathWays)
#new_locuss = calculate_new_locus(pathWays)
#pathWays = second_calculate_direction(new_locuss, pathWays)

In [ ]:
AlignMents = []
while results:
    path = results.pop()
    direction = path.split(',', -1)[2:]
    alignA = A
    alignB = B
    for i in range(0, len(direction)):
        if direction[i] == '2':
            pass
        elif direction[i] == '0':
            alignA = alignA[:i] + '-' + alignA[i:]
        elif direction[i] == '1':
            alignB = alignB[:i] + '-' + alignB[i:]
    AlignMents.append([alignA, alignB])
AlignMents

# 算法的完整实现

In [ ]:
import numpy as np
import pandas as pd
import time
# 伪代码中的参数
a = -1  # gap 罚分 #这个例子是我手算的例子，用来修正算法
b = -6  # mismatch 罚分 #这个例子是我手算的例子，用来修正算法
c = 10  # match 奖励 #这个例子是我手算的例子，用来修正算法

'''
    a = -2  # gap 罚分
    b = -1  # mismatch 罚分
    c = 3  # match 奖励
'''

# 伪代码中的数据
A = "ATCGGGCTACATCGGGCTACGGATCGGGCTACGAAAAAAAAAAAAAAAAAAAAAA"
B = "ATCGGATCGGGCTACGATCGGGCTACGATCGCGTTTTTTTAAAAAGAAAAAAAAGGGGGGGGTGTATTGTA"
#A = "GATTACA" #这个例子是我手算的例子，用来修正算法
#B = "GCATGCG" #这个例子是我手算的例子，用来修正算法
#A = "ATCGGGCTACATCGGGCTACGGATCGGGCTACAATCGGGC"
#B = "ATCGGATCGGGCTACGATCGGGCTACGATCGCGACGATCG"

len_A = len(A)
len_B = len(B)
# 路径变量
pathWays = []
# 路径变量的初始值，矩阵的右下角是
OrignLocus = (len(A), len(B))
# 存储结果
results = []
# 创建大小为（len(A) + 1） x （len(B) + 1）的矩阵
matrix = np.zeros((len_A + 1, len_B + 1))
#matrix = pd.DataFrame(np.zeros((len_A + 1, len_B + 1)), index=['-']+list(A), columns=['-']+list(B))

# 初始化第一行和第一列
for i in range(len_A + 1):
    matrix[i][0] = i * a

for j in range(len_B + 1):
    matrix[0][j] = j * a

# 计算打分值
for i in range(1, len_A + 1, 1):
    # i的取值范围是从1到len(A)+1，在这个例子里是0-6
    for j in range(1, len_B + 1, 1):
        # 计算矩阵(i,j)处的值
        if A[i-1] != B[j-1]:
            # 如果不相等，就是mismatch
            matrix[i][j] = max(matrix[i-1][j] + a, matrix[i][j-1] + a, matrix[i-1][j-1] + b)
        else:
            # 如果相等，就是match
            matrix[i][j] = max(matrix[i-1][j] + a, matrix[i][j-1] + a, matrix[i-1][j-1] + c)
# matrix
# 打印矩阵
print(matrix)


In [ ]:
def calculate_direction(locus):# 基于当前坐标计算方向
    # 方向，每次使用都更新
    i = locus[0]
    j = locus[1]
    calculateDirections = []
    print(i,j)
    # 原来的写法没考虑match或者mismatch的情况，如果matchl，那么一定会产生一个方向2，即calculateDirections.extend('2')
    print(A[i-1],B[j-1])
    if A[i-1] == B[j-1]:
        # 如果相等，就是match
        print(A[i-1],B[j-1])
        calculateDirections.extend('2') # 相等的时候一定可以走对角线
        print('match')
        pass
    else:
        if matrix[i-1][j-1].size > 0 and np.any(matrix[i-1][j-1] + b == matrix[i][j]):
            calculateDirections.extend('2')
        print('mismatch')
    # 检查对应位置的序列A得值是否等于序列B的值
    # 检查左方  左是0
    if matrix[i-1][j].size > 0 and matrix[i-1][j] + a == matrix[i][j]:
        calculateDirections.extend('0')
        print('左移')
    # 检查上方  上是1
    if matrix[i][j-1].size > 0 and matrix[i][j-1] + a == matrix[i][j]:
        calculateDirections.extend('1')
    print(calculateDirections)
    return calculateDirections
# 从pathWays计算新坐标，获得新坐标表列表，第二次计算
def calculate_new_locus(pathWays):
    new_locuss=[]
    Directions = len(pathWays)
    print(f"{'从上游有'}{Directions}{'条路径需要计算'}")
    for path in range(0, Directions): # 索引传入的值（‘方向0’或‘方向1’或‘方向2’）在pathWays中的条目，
        lastDirection = pathWays[path].split(',',-1)
        zero = lastDirection.count('0')# 左移
        one = lastDirection.count('1')# 上移
        two = lastDirection.count('2')# 左上移
        locuss = (int(OrignLocus[0]) - zero - two, int(OrignLocus[1]) - one - two)
        new_locuss.append(locuss)
    print(f"{'从pathway新计算的坐标集合'}{new_locuss}")
    return new_locuss
# 第二次坐标计算
def second_calculate_direction(new_locuss,pathWays):
    pWs = len(new_locuss)#这个时候len(new_locuss)一定等于len(pathWays)
    print(f"{'Total'}{pWs}{'个new locuss'}")#计算结果
    localDirectionss = []
    addPathways= []
    for pW in range(0,pWs):
        # 计算新坐标中的每一个值的方向
        locus_tuple = new_locuss[pW]
        if locus_tuple == (0,0):# locus_tuple坐标为（0，0）的时候，不需要计算方向，直接把这条pathWay加到result中的条目后面。
            results.append(pathWays[pW])
            print(f"{'到达终点后pathway的第'}{pW}{'个'}{'路径写入result'}")#计算结果
            continue #跳出本次循环，进入下一次循环
        else:# 不到终点就继续计算
            localDirections = calculate_direction(locus_tuple)#计算新方向，输出0，1，2，会输出不止一个新方向，
            print(f"{'新方向有'}{localDirections}")#展示计算结果
            for num in range(0,len(localDirections)):
                addPathway = pathWays[pW] + ',' + f"{localDirections[num]}"#形成新的路径
                addPathways.append(addPathway)
    print(f"{'pathWays的计算结果'}{pathWays}")#计算结果
    return addPathways


In [ ]:
%%time
localDirections = calculate_direction(OrignLocus)
R = len(localDirections)
for i in range(R):
    pathway = f"{localDirections[i]}"
    pathWays.append(pathway)
while len(pathWays) != 0:
    new_locuss = calculate_new_locus(pathWays)
    pathWays = second_calculate_direction(new_locuss, pathWays)
AlignMents = []
while results:
    path = results.pop()[::-1]
    direction = path.split(',', -1)
    alignA = A
    alignB = B
    for i in range(0, len(direction)):
        if direction[i] == '2':
            pass
        elif direction[i] == '0':
            alignA = alignA[:i] + '-' + alignA[i:]
        elif direction[i] == '1':
            alignB = alignB[:i] + '-' + alignB[i:]
    AlignMents.append([alignA, alignB])
AlignMents

In [ ]:
pathWays = ['1,1,1,1,1,2,1,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,2,2,0,2,2,2',
 '1,1,1,1,1,2,1,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,2,0,2,2,2,2',
 '1,1,1,1,1,2,1,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0,2,2,2,2,2',
 '1,1,1,1,1,2,1,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,0,0,2,0,0,2,2,0,2,2,2',
 '1,1,1,1,1,2,1,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,0,0,2,0,0,2,0,2,2,2,2',
 '1,1,1,1,1,2,1,2,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,0,0,2,0,0,0,2,2,2,2,2',
 '1,1,1,1,1,2,1,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0,2,2,0,2,2,2',
 '1,1,1,1,1,2,1,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0,2,0,2,2,2,2',
 '1,1,1,1,1,2,1,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0,2,2,2,2,2',
 '1,1,1,1,1,2,1,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,0,0,2,0,0,2,2,0,2,2,2',
 '1,1,1,1,1,2,1,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,0,0,2,0,0,2,0,2,2,2,2',
 '1,1,1,1,1,2,1,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,0,0,2,0,0,0,2,2,2,2,2']

In [ ]:
results

In [ ]:
AlignMents = []
while results:
    path = results.pop()[::-1]
    direction = path.split(',', -1)
    alignA = A
    alignB = B
    for nd in range(0, len(direction)):
        if direction[nd] == '2':
            pass
        elif direction[nd] == '0':
            alignA = alignA[:nd] + '-' + alignA[nd:]
        elif direction[nd] == '1':
            alignB = alignB[:nd] + '-' + alignB[nd:]
    AlignMents.append([alignA, alignB])
AlignMents

# 加速函数实现
- 放弃字符串，整体上使用numba numpy的array
- 使用了 NumPy 的数组切片和 np.arange 来初始化矩阵的第一行和第一列，这比逐个元素赋值更快
将条件判断和分数计算合并为一行来减少代码量。这些更改可以提高代码的效率。

In [7]:
# 定义环境
import numpy as np
import time
from numba import jit,int32,types
from numba.typed import List
#A = "ATCGGGCTACATCGGGCTACGGATCGGGCTACGAAAAAAAAAAAAAAAAAAAAAA"
#B = "ATCGGATCGGGCTACGATCGGGCTACGATCGCGTTTTTTTAAAAAGAAAAAAAAGGGGGGGGTGTATTGTA"
A = str("ATCGGGCTACATCGGGCTACGGATCGGGCTACGAAAAAAAAAAAAAAAAAAAAAA")
B = str("ATCGGATCGGGCTACGATCGGGCTACGATCGCGTTTTTTTAAAAAGAAAAAAAAGGGGGGGGTGTATTGTA")
len_A = int32(len(A))
len_B = int32(len(B))
a = int32(-1)  # gap 罚分 #这个例子是我手算的例子，用来修正算法
b = int32(-6) # mismatch 罚分 #这个例子是我手算的例子，用来修正算法
c = int32(10)  # match 奖励 #这个例子是我手算的例子，用来修正算法
OrignLocus = (len_A, len_B)
left = int32(0)
up = int32(1)
left_up = int32(2)

@jit(nopython=True) # 打分矩阵
def score_matrix(A, B, a, b, c, len_A, len_B):
    matrix = np.zeros((len_A + 1, len_B + 1), dtype=np.int32)
    matrix[:, 0] = np.arange(len_A + 1) * a
    matrix[0, :] = np.arange(len_B + 1) * a
    for i in range(1, len_A + 1):
        for j in range(1, len_B + 1):
            match = A[i-1] == B[j-1]
            score = c if match else b
            matrix[i, j] = max(matrix[i-1, j] + a, matrix[i, j-1] + a, matrix[i-1, j-1] + score)
    return matrix
matrix = score_matrix(A, B, a, b, c, len_A, len_B)

seqA = A
seqB = B
@jit(nopython=True)
def build_match_table(seq1, seq2):
    len_seq1 = len(seq1)
    len_seq2 = len(seq2)
    match_table = np.zeros((len_seq1, len_seq2), dtype=np.bool_)
    for i in range(len_seq1):
        for j in range(len_seq2):
            match_table[i][j] = seq1[i] == seq2[j]
    return match_table
# 构建匹配表
match_table = build_match_table(seqA, seqB)

# 通用函数计算方向
@jit(nopython=True)
def calculate_direction(locus_tuple, a, b, match_table, matrix, left, up, left_up):
    i, j = locus_tuple
    calculateDirections = []
    matrix_i_1_j_1 = matrix[i-1][j-1]
    matrix_i_1_j = matrix[i-1][j]
    matrix_i_j_1 = matrix[i][j-1]
    matrix_i_j = matrix[i][j]
    if match_table[i][j]:
        calculateDirections.append(left_up)  # 相等的时候一定可以走对角线
    else:
        if matrix_i_1_j_1 + b == matrix_i_j:
            calculateDirections.append(left_up)
    if matrix_i_1_j + a == matrix_i_j:
        calculateDirections.append(left)
    if matrix_i_j_1 + a == matrix_i_j:
        calculateDirections.append(up)
    return calculateDirections
# 新坐标
def calculate_new_locus(pathWays):
    new_locuss = np.empty((len(pathWays), 2), dtype=np.int32)
    for idx, path in enumerate(pathWays):
        zero = np.sum(np.where(path == np.int32(0), 1, 0))
        one = np.sum(np.where(path == np.int32(1), 1, 0))
        two = np.sum(np.where(path == np.int32(2), 1, 0))
        locuss = (OrignLocus[0] - zero - two, OrignLocus[1] - one - two)
        new_locuss[idx] = locuss
    return new_locuss

####  打分矩阵

In [11]:
@jit(nopython=True) # 打分矩阵
def score_matrix(A, B, a, b, c, len_A, len_B):
    matrix = np.zeros((len_A + 1, len_B + 1), dtype=np.int32)
    matrix[:, 0] = np.arange(len_A + 1) * a
    matrix[0, :] = np.arange(len_B + 1) * a
    for i in range(1, len_A + 1):
        for j in range(1, len_B + 1):
            match = A[i-1] == B[j-1]
            score = c if match else b
            matrix[i, j] = max(matrix[i-1, j] + a, matrix[i, j-1] + a, matrix[i-1, j-1] + score)
    return matrix


In [ ]:
%%time
matrix = score_matrix(A, B, a, b, c, len_A, len_B)

#### 构建匹配表

In [1]:
%%time
seqA = A
seqB = B
@jit(nopython=True)
def build_match_table(seq1, seq2):
    len_seq1 = len(seq1)
    len_seq2 = len(seq2)
    match_table = np.zeros((len_seq1, len_seq2), dtype=np.bool_)
    for i in range(len_seq1):
        for j in range(len_seq2):
            match_table[i][j] = seq1[i] == seq2[j]
    return match_table
# 构建匹配表
match_table = build_match_table(seqA, seqB)


NameError: name 'A' is not defined

#### 计算方向

In [ ]:
%%time
# 通用函数计算方向
@jit(nopython=True)
def calculate_direction(locus_tuple, a, b, match_table, matrix, left, up, left_up):
    i, j = locus_tuple
    calculateDirections = []
    matrix_i_1_j_1 = matrix[i-1][j-1]
    matrix_i_1_j = matrix[i-1][j]
    matrix_i_j_1 = matrix[i][j-1]
    matrix_i_j = matrix[i][j]
    if match_table[i][j]:
        calculateDirections.append(left_up)  # 相等的时候一定可以走对角线
    else:
        if matrix_i_1_j_1 + b == matrix_i_j:
            calculateDirections.append(left_up)
    if matrix_i_1_j + a == matrix_i_j:
        calculateDirections.append(left)
    if matrix_i_j_1 + a == matrix_i_j:
        calculateDirections.append(up)
    return calculateDirections


查表获得序列匹配结果，

In [ ]:
def preprocess_pathways(pathWays):
    # 将字符串转换为 NumPy 数组
    processed_pathWays = []
    for path in pathWays:
        directions = np.array([int(d) for d in path.split(',')], dtype=np.int32)
        processed_pathWays.append(directions)
    return processed_pathWays

#### 计算新坐标
- 返回一个new_locus，是一个array

In [55]:
%%time
@jit(nopython=True,parallel=True)
def calculate_new_locus(pathWays):
    new_locuss = np.empty((len(pathWays), 2), dtype=np.int32)
    for idx, path in enumerate(pathWays):
        zero = np.sum(np.where(path == np.int32(0), 1, 0))
        one = np.sum(np.where(path == np.int32(1), 1, 0))
        two = np.sum(np.where(path == np.int32(2), 1, 0))
        locuss = (OrignLocus[0] - zero - two, OrignLocus[1] - one - two)
        new_locuss[idx] = locuss
    return new_locuss


CPU times: user 187 µs, sys: 5 µs, total: 192 µs
Wall time: 217 µs


#### 计算坐标循环体

In [6]:

from numba.typed import List
from numba import njit, prange
import numba
addPathways = List()
@jit
def second_calculate_direction(new_locuss, pathWays, a, b, match_table, matrix):
    for pW in range(len(new_locuss)):
        locus_tuple = new_locuss[pW]
        if locus_tuple[0] == 0 and locus_tuple[1] == 0:
            addPathways.append(pathWays[pW])
        else:
            localDirections = calculate_direction(locus_tuple, a, b, match_table, matrix, left, up, left_up)
            for num in range(len(localDirections)):
                addPathway = np.append(pathWays[pW], localDirections[num])
                addPathways.append(addPathway)
    return addPathways

/var/folders/6c/33tr6btx0g32l4wn_9kvdcjw0000gn/T/ipykernel_93166/3981766319.py:5: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [34]:
from multiprocessing import Pool

def process_single_locus(locus_tuple, pathWays, a, b, match_table, matrix, left, up, left_up):
    addPathways = []
    if locus_tuple[0] == 0 and locus_tuple[1] == 0:
        addPathways.append(pathWays[locus_tuple[2]])
    else:
        localDirections = calculate_direction(locus_tuple, a, b, match_table, matrix, left, up, left_up)
        for num in localDirections:
            addPathway = np.append(pathWays[locus_tuple[2]], num)
            addPathways.append(addPathway)
    return addPathways


def second_calculate_direction(new_locuss, pathWays, a, b, match_table, matrix, left, up, left_up):
    #pathWays_list = pathWays.tolist()  # 将 NumPy 数组转换为列表
    with Pool() as pool:
        results = pool.starmap(process_single_locus, [(locus_tuple, pathWays, a, b, match_table, matrix, left, up, left_up) for locus_tuple in new_locuss])
    # 合并所有进程的结果
    addPathways = [item for sublist in results for item in sublist]
    return addPathways


# 调用 second_calculate_direction


#### 定义pathWays

In [30]:
%%time
# 计算新方向并更新路径 # 示例坐标
localDirections = calculate_direction(OrignLocus, a, b, match_table, matrix, left, up, left_up)
R = len(localDirections)
pathWays = []
for i in range(R):
    pathway = int32(localDirections[i])
    pathWays.append(pathway)

CPU times: user 45 µs, sys: 0 ns, total: 45 µs
Wall time: 47.2 µs


#### 循环体

In [28]:
number = 1

In [35]:
%%time
while len(pathWays) != 0:
    print(number)
    new_locuss = calculate_new_locus(pathWays)
    pathWays = second_calculate_direction(new_locuss, pathWays, a, b, match_table, matrix, left, up, left_up)
    number += 1

1


Process SpawnPoolWorker-21:
Process SpawnPoolWorker-22:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/sunhao/anaconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/sunhao/anaconda3/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/sunhao/anaconda3/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/Users/sunhao/anaconda3/lib/python3.11/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'process_single_locus' on <module '__main__' (built-in)>
  File "/Users/sunhao/anaconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/sunhao/anaconda3/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._

KeyboardInterrupt: 